In [45]:
import openai
import traceback
from openai import OpenAI
import os

In [95]:

os.environ['OPENAI_API_KEY']=''

# Function to get completion from GPT-4-turbo or GPT-3.5-turbo
def gpt_agent(prompt, model="gpt-4"):
    client = OpenAI(
              organization='org-SAvFzVyErxizCXaLpvwivG6u',
              project='proj_7Zd4V8jp2r7hMuw2Siyr1Ucz',
            )
    completion = client.chat.completions.create(
          model="gpt-4o",
          messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}]
        )
    print(completion.choices[0].message.content)
    return completion.choices[0].message.content

# Function to clean up code (remove markdown formatting like ```python and `python`)
def clean_code(code):
    # Remove any markdown code fences (```) and 'python' marker
    if code.startswith("```"):
        # Split by ``` and take the code in between, while also stripping 'python' if it appears after ```
        code = code.split("```")[1]
    
    # Remove any remaining 'python' marker if it starts the cleaned block
    code = code.replace("python", "").strip()
    
    return code.strip()

# Access content this way

user_prompt = 'Find me maximum and minimum of column salary from first file. find their college_name from the second file. two files can be joined by column name'

# Load CSV operation prompt for the agent
initial_operation_prompt = f"""
You are to write Python code that loads two CSV files and performs a basic operation (like calculating the mean of a column, filtering rows, etc.).
The CSV file is named 'data.csv' and 'data_ext.csv'.{user_prompt}  Dont provide any instruction just give code, so that it can be executed.
"""

# Function to execute the code dynamically and capture errors
def execute_code(code):
    try:
        exec(code, globals())
        return None  # No errors
    except Exception as e:
        return traceback.format_exc()
    
    # Function for Agent 3 to evaluate the result using LLM
def evaluate_result_with_llm(user_query, code, result):
    evaluation_prompt = f"""
    The following Python code was written to fulfill the user's query:\n\n{user_query}\n\nThe code is:\n\n{code}\n
    The result produced is:\n\n{result}\n
    Please evaluate if the result is correct based on the user query. If it's correct, respond with 'correct'. If it's incorrect, explain what is wrong.
    """
    
    # Agent 3 uses LLM to evaluate if the result matches the query intent
    response = gpt_agent(evaluation_prompt)
    
    return response.strip()

# Function to run the agents in a loop
def run_agents(iterations, operation_prompt):
    for i in range(iterations):
        # Agent 1 generates code
        agent_1_response = gpt_agent(operation_prompt)
        print(f"\nIteration {i+1} - Agent 1's code:\n{agent_1_response}")
        
        # Clean the generated code to remove markdown syntax
        cleaned_code = clean_code(agent_1_response)
        print(cleaned_code)

        # Agent 2 executes the code
        error = execute_code(cleaned_code)

        if error:
            print(f"\nAgent 2 found an error:\n{error}")
            # Feedback to Agent 1 to fix the code
            fix_prompt = f"""
            There was an error in the following Python code:\n\n{agent_1_response}\n\nError details:\n{error}\n
            Please fix the code and try again. Dont provide any instruction just give code, so that it can be executed. If it is a column name error, find the best possible column name and write the code against it.
            """
            operation_prompt = fix_prompt  # Update prompt for next iteration
        else:
            print("\nAgent 2: Code executed successfully!")
            
            # Simulate the result (since we cannot directly execute the code and produce output here)
            actual_output = "Example output from running the code"  # This would be the result of running the Python code in a real environment
            
            # Agent 3 (LLM-based) evaluates the result
            agent_3_evaluation = evaluate_result_with_llm(user_prompt, cleaned_code, actual_output)
            
            print(f"\nAgent 3's evaluation of the result: {agent_3_evaluation}")
            
            if agent_3_evaluation.lower() == "correct":
                print("\nAgent 3: The result is correct!")
                break
            else:
                print(f"\nAgent 3: The result is incorrect! {agent_3_evaluation}")
                # Feedback to Agent 1 to rewrite the code based on incorrect result
                fix_prompt = f"""
                The following Python code produced an incorrect result:\n\n{cleaned_code}\n
                The result was incorrect because:\n\n{agent_3_evaluation}\n
                Please rewrite the code to produce the correct result. Dont provide any instruction just give code, so that it can be executed.
                """
                operation_prompt = fix_prompt  # Update prompt for next iteration

In [96]:
# Set number of iterations and run the autonomous loop
n_iterations = 5
run_agents(n_iterations, initial_operation_prompt)

```python
import pandas as pd

# Load the CSV files
df1 = pd.read_csv('data.csv')
df2 = pd.read_csv('data_ext.csv')

# Find maximum and minimum salary in the first file
max_salary = df1['salary'].max()
min_salary = df1['salary'].min()

# Get ID or Name of the person with max and min salary to join with the second file
max_salary_person = df1[df1['salary'] == max_salary]['name'].values[0]
min_salary_person = df1[df1['salary'] == min_salary]['name'].values[0]

# Join with the second file to get college_name
max_college_name = df2[df2['name'] == max_salary_person]['college_name'].values[0]
min_college_name = df2[df2['name'] == min_salary_person]['college_name'].values[0]

print(f"The college name of the person with the max salary ({max_salary}) is: {max_college_name}")
print(f"The college name of the person with the min salary ({min_salary}) is: {min_college_name}")
```

Iteration 1 - Agent 1's code:
```python
import pandas as pd

# Load the CSV files
df1 = pd.read_csv('data.csv')
df2 = p

```python
import pandas as pd

# Load the CSV files
df1 = pd.read_csv('data.csv')
df2 = pd.read_csv('data_ext.csv')

# Find maximum and minimum salary in the first file
max_salary = df1['salary'].max()
min_salary = df1['salary'].min()

# Ensure 'name' is a unique identifier before proceeding
df1_unique = df1.drop_duplicates(subset='name')

# Get ID or Name of the person with max and min salary to join with the second file
max_salary_person = df1_unique[df1_unique['salary'] == max_salary]['name'].values[0]
min_salary_person = df1_unique[df1_unique['salary'] == min_salary]['name'].values[0]

# Join with the second file to get college_name using a safer approach
max_college_name = df2[df2['name'] == max_salary_person]['college_name'].values[0]
min_college_name = df2[df2['name'] == min_salary_person]['college_name'].values[0]

print(f"The college name of the person with the max salary ({max_salary}) is: {max_college_name}")
print(f"The college name of the person with the min salary ({min_s

```python
import pandas as pd

# Load the CSV files
df1 = pd.read_csv('data.csv')
df2 = pd.read_csv('data_ext.csv')

# Find maximum and minimum salary in the first file
max_salary = df1['salary'].max()
min_salary = df1['salary'].min()

# Ensure 'name' is a unique identifier before proceeding
df1_unique = df1.drop_duplicates(subset='name')

# Get ID or Name of the person with max and min salary to join with the second file
max_salary_person = df1_unique.loc[df1_unique['salary'] == max_salary, 'name'].values
min_salary_person = df1_unique.loc[df1_unique['salary'] == min_salary, 'name'].values

# Verify if persons are found for max and min salary to avoid index error
if max_salary_person.size > 0 and min_salary_person.size > 0:
    # Join with the second file to get college_name using a safer approach
    person_max = df2.loc[df2['name'] == max_salary_person[0], 'college_name']
    person_min = df2.loc[df2['name'] == min_salary_person[0], 'college_name']

    if not person_max.empty:
    

```python
import pandas as pd

# Load the CSV files
df1 = pd.read_csv('data.csv')
df2 = pd.read_csv('data_ext.csv')

# Find maximum and minimum salary in the first file
max_salary = df1['salary'].max()
min_salary = df1['salary'].min()

# Ensure 'name' is a unique identifier before proceeding
df1_unique = df1.drop_duplicates(subset='name')

# Get ID or Name of the person with max and min salary to join with the second file
max_salary_person = df1_unique.loc[df1_unique['salary'] == max_salary, 'name'].values
min_salary_person = df1_unique.loc[df1_unique['salary'] == min_salary, 'name'].values

# Verify if persons are found for max and min salary to avoid index error
if max_salary_person.size > 0 and min_salary_person.size > 0:
    # Join with the second file to get college_name using a safer approach
    person_max = df2.loc[df2['name'].isin(max_salary_person), 'college_name']
    person_min = df2.loc[df2['name'].isin(min_salary_person), 'college_name']

    if not person_max.empty:
    